# 1.F. Collection -- Reviews -- RapReviews

ALMOST THERE. Ok, so Pitchfork has been taken care of. **RapReviews.com** is another popular review site. One reason we want to grab content from this site, is that the site has a well formatted archive of older albums that we are unlikely to pick up from other data sources (NeedleDrop is newer, Pitchfork doesn't cover oolder material as much. They unfortunately do not have an API. With that said, the workflow in this notebook is as follows:

 1. Use **BeautifulSoup** to grab everything from our archive page and extract each album review URL
 2. Enter into each album review page and extract the text of the review by isolating for **<p>** tags and **font** tags for scores.

There was some valuable information here beyond text (*vibe score, lyric score*). We did not leverage this information, but we did capture it. Something to consider if you're tackling this yourself.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

In [152]:
#Build our request for the archive page and grab the soup
url = 'http://www.rapreviews.com/all.html'
res = requests.get(url)
rr_soup = BeautifulSoup(res.content)

C:\Users\Owen\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Owen\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [153]:
#We can build a list of the archive album page IDs
archived_album_review_soup_list = rr_soup.find_all("a", href=lambda href: href and "archive" in href)

In [154]:
#Generate a queryable list with those page IDs
archive_url_list = []
for element in archived_album_review_soup_list:
    archive_url_list.append('http://www.rapreviews.com/' + element['href'])

In [159]:
#Ready our lists to append our album information to
archive_review_text_list = []
archive_review_title_list = []

#For every archive URL
for url in archive_url_list:
    
    #Pause, and then grab our soup
    time.sleep(.5)
    try:
        res = requests.get(url)
        rr_soup = BeautifulSoup(res.content)
    
        #Grab paragraphs if they are there.
        try:
            archive_review_text_list.append(rr_soup.find_all('p'))
            
            #If we are seeing paragraphs (that is, the page has text), try grabbing the title/score information
            try:       
                archive_review_title_list.append(rr_soup.find_all('font')[0].contents)
            except:
                archive_review_title_list.append('')
                
        #Skip everything if there's a problem       
        except:
            archive_review_title_list.append('')
            archive_review_text_list.append('')

       
            
    except:
        #Skip everything if there's a problem
        archive_review_text_list.append('')
        archive_review_title_list.append('')
    
    

In [160]:
len(archive_review_text_list), len(archive_review_title_list)

(3783, 3783)

In [172]:
#Make our dataframe
temp_storage_df = pd.DataFrame({'review_text':archive_review_text_list, 'review_title':archive_review_title_list })

In [204]:
#check out our dataframe
pd.set_option('display.max_rows', None) 
temp_storage_df

,review_text,review_title
0,"[<p>\nFor a hip-hop act, the fact that they're...",\nThe 100th Monkey :: Looking at My World :: N...
1,"[<p>\n""And for those of you who don't understa...",\n10BASS T :: Do You Know the Way? :: San Jose...
2,"[<p>\nLouisville, Kentucky is a city that gets...",\n10 Crown :: The Tenth District :: Mastermind...
3,"[<p>\n<a href=""http://www.asideworldwide.com/r...",\n14KT :: Nowalataz :: A-Side Worldwide\n
4,"[<p>\n<a href=""http://www.amazon.com/gp/produc...",\nThe 1978ers (yU &amp; Slimkat) :: P.O.T. EP ...
5,"[<p>\n<a href=""http://www.amazon.com/gp/produc...",\nThe 1978ers :: People of Today :: Mello Musi...
6,"[<p>\n<a href=""http://the2020project.bandcamp....",\nThe 20/20 Project :: Employees of the Year E...
7,"[<p><a href=""http://www.amazon.com/gp/product/...",
8,"[<p><a href=""https://www.amazon.com/gp/product...",
9,"[<p>\n<a href=""https://www.amazon.com/gp/produ...",\n21 Savage x Offset x Metro Boomin :: Without...


In [173]:
#Save what we have currently
temp_storage_df.to_csv('./rapreview/temp_file_3738_entries.csv', index=False)


### Clean Title

In [308]:
#We can grab the artist and album through the title string. these values are delineated with '::'
temp_storage_df = pd.concat([temp_storage_df,temp_storage_df.review_title.str.split("::",expand=True)], axis=1)

In [309]:
#rename our columns
temp_storage_df.columns = ['review_text', 'review_title', 'artist_name', 'album_name', 'label_name']

In [310]:
#replace gunk for artist name including newlines, ampersand issues, and double spaces
temp_storage_df['artist_name'] = temp_storage_df['artist_name'].str.replace("\n", "").str.strip().str.replace("&amp;", " ").str.replace("  ", "")

In [311]:
#replace gunk for album including newlines, ampersand issues, and double spaces
temp_storage_df['album_name'] = temp_storage_df['album_name'].str.replace("\n", "").str.strip().str.replace("&amp;", " ").str.replace("  ", "")

In [312]:
#Drop NAs
temp_storage_df = temp_storage_df.dropna()

In [313]:
#Let's see how we're doing so far.
temp_storage_df.tail()

,review_text,review_title,artist_name,album_name,label_name
3777,"[<p>\nExhortations to put down your mouse, pic...",\nLyrics Born :: Overnite Encore: Lyrics Born ...,Lyrics Born,Overnite Encore: Lyrics Born Live!,Quannum Projects\n
3779,"[<p>\n<a href=""http://www.amazon.com/gp/produc...",\nLyrics Born :: As U Were :: Mobile Home Reco...,Lyrics Born,As U Were,Mobile Home Recordings/Decon Records\n
3780,"[<p>\n<a href=""http://www.amazon.com/gp/produc...",\nLyrics Born &amp; The LB Mixed Re-View :: Re...,Lyrics Born The LB Mixed Re-View,Real People,Mobile Home Recordings\n
3781,"[<p>\n<a href=""http://www.amazon.com/gp/produc...","\nLyrics Born :: Now Look What You've Done, Ly...",Lyrics Born,"Now Look What You've Done, Lyrics Born! Greate...",Mobile Home Recordings\n
3782,[<p>\nHorrific rhymes and concepts have always...,\nLyrikal Additivez :: Raw Rugged Underground ...,Lyrikal Additivez,Raw Rugged Underground,Kill Yourself Records\n


### Clean Review

In [314]:
#Font Color is the way that we can break out the scores from the review text.
temp_storage_df = pd.concat([temp_storage_df,temp_storage_df.review_text.str.split("<font color=",expand=True)], axis=1)

In [316]:
temp_storage_df.head()

,review_text,review_title,artist_name,album_name,label_name,0,3
0,"[<p>\nFor a hip-hop act, the fact that they're...",\nThe 100th Monkey :: Looking at My World :: N...,The 100th Monkey,Looking at My World,Narcofunk Records\n,"[<p>\nFor a hip-hop act, the fact that they're...","""red""><b>TOTAL</b> Vibes</font>: <b>7 of 10</b..."
1,"[<p>\n""And for those of you who don't understa...",\n10BASS T :: Do You Know the Way? :: San Jose...,10BASS T,Do You Know the Way?,San Jose Sound Recording Company\n,"[<p>\n""And for those of you who don't understa...",None
2,"[<p>\nLouisville, Kentucky is a city that gets...",\n10 Crown :: The Tenth District :: Mastermind...,10 Crown,The Tenth District,Mastermind Productions\n,"[<p>\nLouisville, Kentucky is a city that gets...","""red""><b>TOTAL</b> Vibes</font>: <b>4 of 10</b..."
3,"[<p>\n<a href=""http://www.asideworldwide.com/r...",\n14KT :: Nowalataz :: A-Side Worldwide\n,14KT,Nowalataz,A-Side Worldwide\n,"[<p>\n<a href=""http://www.asideworldwide.com/r...","""red""><b>TOTAL</b> Vibes</font>: <b>8 of 10</b..."
4,"[<p>\n<a href=""http://www.amazon.com/gp/produc...",\nThe 1978ers (yU &amp; Slimkat) :: P.O.T. EP ...,The 1978ers (yU Slimkat),P.O.T. EP,Mello Music Group\n,"[<p>\n<a href=""http://www.amazon.com/gp/produc...","""red""><b>TOTAL</b> Vibes</font>: <b>7 of 10</b..."


In [327]:
#Clean up our review. That includes brackets, dropping tags, extra spaces, and our weird column name
temp_storage_df[0] = temp_storage_df[0].str.replace("<[^>]*>", "").str.replace('\\xa0','').str.replace('\\n',' ').str.replace('\\\'s','').str.replace('\\\'','')
temp_storage_df[0] = temp_storage_df[0].str.replace("[", "").str.replace(']','').str.replace(')',' ').str.replace('(','')
temp_storage_df[0] = temp_storage_df[0].str.replace(' , ',"").replace('/\s\s+/g', '').str.replace('  '," ")
temp_storage_df = temp_storage_df.rename(columns={0:'cleaned_text'})

In [330]:
#Just save the columns that matter to us. score data proved to be a bit more challenging -- maybe next time
temp_storage_df = temp_storage_df[['artist_name', 'album_name', 'cleaned_text']]

temp_storage_df.to_csv('./rapreview/ready_to_combine.csv', index=False)